Value based funcitons are based on the Bellman Equations.
- $V^*(s) = \underset{a}{\text{max}} \sum_{s'} P(s'|s,a)(R(s,a) + \gamma V^*(s'))$
- $Q^*(s,a) =  \sum_{s'} P(s'|s,a)[R(s,a) + \gamma \underset{a}{\text{max}} Q^*(s'])$

And for completion- the deterministic versions
- $V^*(s) = \underset{a}{\text{max}}(R(s,a) + \gamma\ V^*(s’))$
- $Q^*(s,a) = R(s,a)  + \gamma\  \underset{a'}{\text{max}}\ Q^*(s’,a'))$

The goal is to optimize for expected reward which is defined as follows
- $R = E_{\tau \sim p_\theta(\tau)}[\sum_t R(s_t,a_t)]$

If we know the transitions for all states (and there's a small number of them), we can just do policy iteration:
- Evaluate best possible next action given state and current value function, take that action
- Update value function based on observed reward

Since we know all the transitions and states, we can essentialy just fill out a table of the values we know
import numpy as np


In [12]:
import random

decay = 0.99
maximum = 10**(-3)

action_len = 4
actions = [(1, 0), (0, -1), (-1, 0), (0, 1)] # Down, Left, Up, Right
row_len = 3
col_len = 4
utility = [[0, 0, 0, 1], [0, 0, 0, -1], [0, 0, 0, 0], [0, 0, 0, 0]]
policy = [[random.randint(0, 3) for j in range(col_len)] for i in range(row_len)] # construct a random policy

# Visualization
def printEnvironment(arr, policy=False):
    res = ""
    for r in range(row_len):
        res += "|"
        for c in range(col_len):
            if r == c == 1:
                val = "WALL"
            elif r <= 1 and c == 3:
                val = "+1" if r == 0 else "-1"
            else:
                val = ["Down", "Left", "Up", "Right"][arr[r][c]]
            res += " " + val[:5].ljust(5) + " |" # format
        res += "\n"
    print(res)


def action_utility(utility, r, c, action):
    dr, dc = actions[action]
    newR, newC = r+dr, c+dc
    if newR < 0 or newC < 0 or newR >= row_len or newC >= col_len or (newR == newC == 1):
        return utility[r][c]
    else:
        return utility[newR][newC]


def get_utilities(utility, r, c, action):
    u = 0
    u += 0.1 * decay * action_utility(utility, r, c, (action-1)%4)
    u += 0.8 * decay * action_utility(utility, r, c, action)
    u += 0.1 * decay * action_utility(utility, r, c, (action+1)%4)
    return u

def policyEvaluation(policy, utility):
    while True:
        next_utility = [[0, 0, 0, 1], [0, 0, 0, -1], [0, 0, 0, 0], [0, 0, 0, 0]]
        error = 0
        for r in range(row_len):
            for c in range(col_len):
                if (r <= 1 and c == 3) or (r == c == 1):
                    continue
                next_utility[r][c] = get_utilities(utility, r, c, policy[r][c]) 
                error = max(error, abs(next_utility[r][c]-utility[r][c]))
        utility = next_utility
        if error < maximum * (1-decay) / decay:
            break
    return utility

def policyIteration(policy, utility):
    while True:
        utility = policyEvaluation(policy, utility)
        unchanged = True
        for r in range(row_len):
            for c in range(col_len):
                if (r <= 1 and c == 3) or (r == c == 1):
                    continue
                maxAction, max_util = None, -float("inf")
                for action in range(action_len):
                    u = get_utilities(utility, r, c, action)
                    if u > max_util:
                        maxAction, max_util = action, u
                if max_util > get_utilities(utility, r, c, policy[r][c]):
                    policy[r][c] = maxAction # the action that maximizes the utility
                    unchanged = False
        if unchanged:
            break
        printEnvironment(policy)
    return policy

print("The initial random policy is:\n")
printEnvironment(policy)
# Policy iteration
policy = policyIteration(policy, utility)
# Print the optimal policy
print("The optimal policy is:\n")
printEnvironment(policy)

The initial random policy is:

| Left  | Down  | Up    | +1    |
| Right | WALL  | Up    | -1    |
| Down  | Right | Up    | Left  |

| Right | Right | Right | +1    |
| Down  | WALL  | Left  | -1    |
| Right | Right | Up    | Down  |

| Right | Right | Right | +1    |
| Up    | WALL  | Left  | -1    |
| Right | Right | Up    | Down  |

| Right | Right | Right | +1    |
| Up    | WALL  | Left  | -1    |
| Up    | Right | Up    | Down  |

| Right | Right | Right | +1    |
| Up    | WALL  | Left  | -1    |
| Up    | Left  | Up    | Down  |

| Right | Right | Right | +1    |
| Up    | WALL  | Left  | -1    |
| Up    | Left  | Left  | Down  |

The optimal policy is:

| Right | Right | Right | +1    |
| Up    | WALL  | Left  | -1    |
| Up    | Left  | Left  | Down  |

